In [10]:
import cv2
import numpy as np

def find_and_warp_chessboard(image_path, debug=False):
    """
    Reads an image, detects the largest 4-sided contour (chessboard),
    and returns a warped (top-down) view of the board.
    """

    # 1. Read and preprocess
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Could not read image from {image_path}")
    
    orig = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 2. Edge detection
    #    Adjust the thresholds if needed
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    # 3. Find contours
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 4. Sort contours by area (largest first)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # 5. Look for the largest 4-cornered contour
    board_contour = None
    for cnt in contours:
        # Approximate the contour to a polygon
        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02 * perimeter, True)

        # If we find a 4-sided polygon, assume it's the board
        if len(approx) == 4:
            board_contour = approx
            break

    if board_contour is None:
        raise ValueError("No 4-sided chessboard contour could be found.")

    # 6. Reorder the corners for consistent perspective transform
    pts = board_contour.reshape(4, 2).astype("float32")
    rect = np.zeros((4, 2), dtype="float32")

    # Sort by sum (top-left, bottom-right)
    s = np.sum(pts, axis=1)
    rect[0] = pts[np.argmin(s)]  # top-left
    rect[2] = pts[np.argmax(s)]  # bottom-right

    # Sort by difference (top-right, bottom-left)
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # top-right
    rect[3] = pts[np.argmax(diff)]  # bottom-left

    (tl, tr, br, bl) = rect

    # 7. Compute the width and height of the board
    widthA = np.linalg.norm(br - bl)
    widthB = np.linalg.norm(tr - tl)
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.linalg.norm(tr - br)
    heightB = np.linalg.norm(tl - bl)
    maxHeight = max(int(heightA), int(heightB))

    # 8. Destination points for perspective transform
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]
    ], dtype="float32")

    # 9. Perspective transform
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(orig, M, (maxWidth, maxHeight))

    # Debug visualization
    if debug:
        cv2.drawContours(orig, [board_contour], -1, (0, 255, 0), 3)
        cv2.imshow("Detected Board Contour", orig)
        cv2.imshow("Warped Board", warped)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return warped

def segment_chessboard(warped_board, debug=False):
    """
    Given a warped top-down image of a chessboard,
    split it into 8x8 individual square images.
    Returns a list of lists (8x8) of squares.
    """
    # Height and width of the warped board
    h, w = warped_board.shape[:2]

    # Each square's dimension (assuming a perfect 8x8 board)
    square_h = h // 8
    square_w = w // 8

    squares = []
    for row in range(8):
        row_squares = []
        for col in range(8):
            y1 = row * square_h
            y2 = (row + 1) * square_h
            x1 = col * square_w
            x2 = (col + 1) * square_w
            square = warped_board[y1:y2, x1:x2]
            row_squares.append(square)

            if debug:
                cv2.imshow(f"Square {row},{col}", square)
                cv2.waitKey(50)
        squares.append(row_squares)

    if debug:
        cv2.destroyAllWindows()

    return squares

if __name__ == "__main__":
    image_path = "chessboard.jpg"

    # 1. Find and warp the board
    warped_board = find_and_warp_chessboard(image_path, debug=True)

    # 2. Segment into squares
    squares = segment_chessboard(warped_board, debug=True)

    # squares is now a list of 8 rows, each containing 8 square images.
    # You can proceed with piece detection or further processing here.
